# Introduction
Gluten intolerance and other gluten-related sensitivities are becoming more and more prevalent. This is already reflected in the increase of gluten-free products offered in supermarkets. However, in everyday-life, gluten-sensitive people are still confronted with several difficulties. They can't just go to the bakery to buy a snack like almost every other person and they always have to take care not to accidentally eat gluten-containing food in restaurants. Ordering a Pizza is basically impossible, the very most pasta is wheat-based and most sauces and gravy are prepared using flour. Therefore, I am going to check Foursquare for restaurants that offer gluten-free dishes or are dedicated gluten-free in the city of Hamburg, Germany. Analyses of the Neighborhoods with numbers of restaurants in general and those offering gluten-free dishes will be of interest for persons that would want to open a dedicated gluten free restaurant and investors.

# Data
I will use publicly available data to create a dataframe of the neighborhoods of Hamburg (Wikipedia). This will be used to screen for gluten-free restaurants in Hamburg and screen for food places all together using the four-square API. Then I will be hopefully able to decide if or where to set up a restaurant.

In [1]:
from bs4 import BeautifulSoup as BSp
import lxml
#import html5lib alternative to lxml
import requests
import pandas as pd

Since I want to find out about gluten-free restaurants in Hamburg, I need a dataframe with all neighborhoods and coordinates first.
There is a Wikipedia site about Hamburg containing all its neighborhoods. I didn't find a site combining the neighborhoods and coordinates. So I'll need to find a work-around for that later. 

### 1. Get text from wikipedia website for scraping

In [2]:
# Import website from URL
URL = 'https://de.wikipedia.org/wiki/Bezirke_in_Hamburg'
source = requests.get(URL).text
soup = BSp(source, 'lxml')
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="de">
 <head>
  <meta charset="utf-8"/>
  <title>
   Bezirke in Hamburg – Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"Bezirke_in_Hamburg","wgTitle":"Bezirke in Hamburg","wgCurRevisionId":192662115,"wgRevisionId":192662115,"wgArticleId":3772,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Bezirk von Hamburg","Politik (Hamburg)","Liste (Ortsteile)","Liste (Stadtbezirke)"],"wgBreakFrames":!1,"wgPageContentLanguage":"de","wgPageContentModel":"wikitext","wgSeparatorTransformTable":[",\t.",".\t,"],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","Januar","Februar","März","April","Mai","Juni","Juli","August","September","Oktober","November","Dezember"],"wgMonthNamesShort":["","Jan.","Feb.","Mär.","Apr.","Mai",

### 2. Scraping
- find table
- extract all rows
- produce a clean list through spliting and slicing

In [3]:
#isolating table
table = soup.find('table', class_='wikitable zebra zebra')
rows = table.tbody.text
rows

'\nKreis\nStadtteile\nAnmerkung\n\n\n1\nLokstedt, Schnelsen, Niendorf, Langenhorn, Fuhlsbüttel, Ohlsdorf, Alsterdorf, Groß Borstel, Winterhude, Eppendorf, Hoheluft (Ost+West!)\n= äußere Stadtteile der heutigen Bezirke Eimsbüttel und Hamburg-Nord\n\n\n2\nEimsbüttel, Harvestehude, Rotherbaum\n= „Kerngebiet“ des späteren Bezirks Eimsbüttel\n\n\n3\nSt. Pauli, Neustadt, Altstadt, Steinwerder, Waltershof, Finkenwerder\n= westlicher Bereich von Hamburg-Mitte\n\n\n4\nSt. Georg, Borgfelde, Hammerbrook, Klostertor, Kleiner Grasbrook, Veddel, Rothenburgsort, Billwerder Ausschlag, Billbrook\n= mittlerer Bereich von Hamburg-Mitte\n\n\n5\nHohenfelde, Eilbek, Hamm, Horn, Billstedt\n= östlicher Bereich von Hamburg-Mitte\n\n\n6\nBarmbek, Dulsberg, Uhlenhorst\n= südlicher Bereich von Hamburg-Nord ohne Hohenfelde\n\n\n7\nAltona, Ottensen, Bahrenfeld, Eidelstedt, Stellingen, Groß Flottbek, Othmarschen, Nienstedten, Blankenese, Sülldorf, Rissen, Osdorf, Lurup\n= heutiger Bezirk Altona zzgl. Eidelstedt und 

In [4]:
# create list by spliting the sting
rowList = rows.split('\n')
rowList

['',
 'Kreis',
 'Stadtteile',
 'Anmerkung',
 '',
 '',
 '1',
 'Lokstedt, Schnelsen, Niendorf, Langenhorn, Fuhlsbüttel, Ohlsdorf, Alsterdorf, Groß Borstel, Winterhude, Eppendorf, Hoheluft (Ost+West!)',
 '= äußere Stadtteile der heutigen Bezirke Eimsbüttel und Hamburg-Nord',
 '',
 '',
 '2',
 'Eimsbüttel, Harvestehude, Rotherbaum',
 '= „Kerngebiet“ des späteren Bezirks Eimsbüttel',
 '',
 '',
 '3',
 'St. Pauli, Neustadt, Altstadt, Steinwerder, Waltershof, Finkenwerder',
 '= westlicher Bereich von Hamburg-Mitte',
 '',
 '',
 '4',
 'St. Georg, Borgfelde, Hammerbrook, Klostertor, Kleiner Grasbrook, Veddel, Rothenburgsort, Billwerder Ausschlag, Billbrook',
 '= mittlerer Bereich von Hamburg-Mitte',
 '',
 '',
 '5',
 'Hohenfelde, Eilbek, Hamm, Horn, Billstedt',
 '= östlicher Bereich von Hamburg-Mitte',
 '',
 '',
 '6',
 'Barmbek, Dulsberg, Uhlenhorst',
 '= südlicher Bereich von Hamburg-Nord ohne Hohenfelde',
 '',
 '',
 '7',
 'Altona, Ottensen, Bahrenfeld, Eidelstedt, Stellingen, Groß Flottbek, Othma

In [5]:
#remove table header objects from list
rowList = rowList[7:]
rowList

['Lokstedt, Schnelsen, Niendorf, Langenhorn, Fuhlsbüttel, Ohlsdorf, Alsterdorf, Groß Borstel, Winterhude, Eppendorf, Hoheluft (Ost+West!)',
 '= äußere Stadtteile der heutigen Bezirke Eimsbüttel und Hamburg-Nord',
 '',
 '',
 '2',
 'Eimsbüttel, Harvestehude, Rotherbaum',
 '= „Kerngebiet“ des späteren Bezirks Eimsbüttel',
 '',
 '',
 '3',
 'St. Pauli, Neustadt, Altstadt, Steinwerder, Waltershof, Finkenwerder',
 '= westlicher Bereich von Hamburg-Mitte',
 '',
 '',
 '4',
 'St. Georg, Borgfelde, Hammerbrook, Klostertor, Kleiner Grasbrook, Veddel, Rothenburgsort, Billwerder Ausschlag, Billbrook',
 '= mittlerer Bereich von Hamburg-Mitte',
 '',
 '',
 '5',
 'Hohenfelde, Eilbek, Hamm, Horn, Billstedt',
 '= östlicher Bereich von Hamburg-Mitte',
 '',
 '',
 '6',
 'Barmbek, Dulsberg, Uhlenhorst',
 '= südlicher Bereich von Hamburg-Nord ohne Hohenfelde',
 '',
 '',
 '7',
 'Altona, Ottensen, Bahrenfeld, Eidelstedt, Stellingen, Groß Flottbek, Othmarschen, Nienstedten, Blankenese, Sülldorf, Rissen, Osdorf, L

In [6]:
# select 'Bezirke'= Neighborhoods from the list
rowList1 = rowList[0:rowList.index('9')-4:5]
rowList2 =rowList[rowList.index('9')+1:-2:5] #there is an inconsistency in the html-table, so two steps are necessary

#combine both lists
rowList3 = rowList1 + rowList2
rowList3

['Lokstedt, Schnelsen, Niendorf, Langenhorn, Fuhlsbüttel, Ohlsdorf, Alsterdorf, Groß Borstel, Winterhude, Eppendorf, Hoheluft (Ost+West!)',
 'Eimsbüttel, Harvestehude, Rotherbaum',
 'St. Pauli, Neustadt, Altstadt, Steinwerder, Waltershof, Finkenwerder',
 'St. Georg, Borgfelde, Hammerbrook, Klostertor, Kleiner Grasbrook, Veddel, Rothenburgsort, Billwerder Ausschlag, Billbrook',
 'Hohenfelde, Eilbek, Hamm, Horn, Billstedt',
 'Barmbek, Dulsberg, Uhlenhorst',
 'Altona, Ottensen, Bahrenfeld, Eidelstedt, Stellingen, Groß Flottbek, Othmarschen, Nienstedten, Blankenese, Sülldorf, Rissen, Osdorf, Lurup',
 'Wilhelmsburg, Georgswerder, Moorwerder, Harburg, Heimfeld, Eißendorf, Wilstorf, Marmstorf, Rönneburg, Langenbeck, Sinstorf, Neuland, Gut Moor, Moorburg, Francop, Altenwerder, Fischbeck, Neugraben, Cranz, Neuenfelde',
 'Lohbrügge, Bergedorf, Altengamme, Curslack, Kirchwerder, Neuengamme, Allermöhe, Billwerder, Moorfleet,   Ochsenwerder, Reitbrook, Spadenland, Tatenberg',
 'Wandsbek, Marienthal

In [7]:
# produce correctly splitted list
Stadtteile = []
for word in rowList3:
    word = word.split(",")
    Stadtteile.extend(word)

Stadtteile

['Lokstedt',
 ' Schnelsen',
 ' Niendorf',
 ' Langenhorn',
 ' Fuhlsbüttel',
 ' Ohlsdorf',
 ' Alsterdorf',
 ' Groß Borstel',
 ' Winterhude',
 ' Eppendorf',
 ' Hoheluft (Ost+West!)',
 'Eimsbüttel',
 ' Harvestehude',
 ' Rotherbaum',
 'St. Pauli',
 ' Neustadt',
 ' Altstadt',
 ' Steinwerder',
 ' Waltershof',
 ' Finkenwerder',
 'St. Georg',
 ' Borgfelde',
 ' Hammerbrook',
 ' Klostertor',
 ' Kleiner Grasbrook',
 ' Veddel',
 ' Rothenburgsort',
 ' Billwerder Ausschlag',
 ' Billbrook',
 'Hohenfelde',
 ' Eilbek',
 ' Hamm',
 ' Horn',
 ' Billstedt',
 'Barmbek',
 ' Dulsberg',
 ' Uhlenhorst',
 'Altona',
 ' Ottensen',
 ' Bahrenfeld',
 ' Eidelstedt',
 ' Stellingen',
 ' Groß Flottbek',
 ' Othmarschen',
 ' Nienstedten',
 ' Blankenese',
 ' Sülldorf',
 ' Rissen',
 ' Osdorf',
 ' Lurup',
 'Wilhelmsburg',
 ' Georgswerder',
 ' Moorwerder',
 ' Harburg',
 ' Heimfeld',
 ' Eißendorf',
 ' Wilstorf',
 ' Marmstorf',
 ' Rönneburg',
 ' Langenbeck',
 ' Sinstorf',
 ' Neuland',
 ' Gut Moor',
 ' Moorburg',
 ' Francop',
 ' A

In [8]:
# 'Hohelust (Ost + West!)' are two Neighborhoods that need to be seperated
# so I remove 'Hohelust (Ost + West!)' and add the actuall names to the end of the list
del(Stadtteile[10])
Stadtteile.append('Hoheluft-Ost')
Stadtteile.append('Hoheluft-West')

# use a list comprehension to remove spaces from list-objects
Stadtteile = [x.strip(' ') for x in Stadtteile]

Stadtteile

['Lokstedt',
 'Schnelsen',
 'Niendorf',
 'Langenhorn',
 'Fuhlsbüttel',
 'Ohlsdorf',
 'Alsterdorf',
 'Groß Borstel',
 'Winterhude',
 'Eppendorf',
 'Eimsbüttel',
 'Harvestehude',
 'Rotherbaum',
 'St. Pauli',
 'Neustadt',
 'Altstadt',
 'Steinwerder',
 'Waltershof',
 'Finkenwerder',
 'St. Georg',
 'Borgfelde',
 'Hammerbrook',
 'Klostertor',
 'Kleiner Grasbrook',
 'Veddel',
 'Rothenburgsort',
 'Billwerder Ausschlag',
 'Billbrook',
 'Hohenfelde',
 'Eilbek',
 'Hamm',
 'Horn',
 'Billstedt',
 'Barmbek',
 'Dulsberg',
 'Uhlenhorst',
 'Altona',
 'Ottensen',
 'Bahrenfeld',
 'Eidelstedt',
 'Stellingen',
 'Groß Flottbek',
 'Othmarschen',
 'Nienstedten',
 'Blankenese',
 'Sülldorf',
 'Rissen',
 'Osdorf',
 'Lurup',
 'Wilhelmsburg',
 'Georgswerder',
 'Moorwerder',
 'Harburg',
 'Heimfeld',
 'Eißendorf',
 'Wilstorf',
 'Marmstorf',
 'Rönneburg',
 'Langenbeck',
 'Sinstorf',
 'Neuland',
 'Gut Moor',
 'Moorburg',
 'Francop',
 'Altenwerder',
 'Fischbeck',
 'Neugraben',
 'Cranz',
 'Neuenfelde',
 'Lohbrügge',
 'B

 ## Get Coordinates for Hamburg area from geoJSON
 
Now that I have a clean list with all Neighborhoods I need their coordinates. I decided to use geocoder to retrieve all coordinates.

In [9]:
import geocoder

#test functionality with the first entry
g = geocoder.osm(Stadtteile[0] + ', Hamburg')
g.osm

{'x': 9.9566624,
 'y': 53.6023971,
 'addr:city': 'Hamburg',
 'addr:state': 'Hamburg',
 'addr:country': 'Deutschland',
 'addr:postal': '22529'}

In [10]:
# generate lat and lng lists and loop through Stadtteile to fill them
# I introduce an exception catch in case geocoder comes up empty with some neighborhoods

Latitude = []
Longitude = []
s = len(Stadtteile)
j = 0

while j < s:
    
    g = geocoder.osm(Stadtteile[j] + ', Hamburg')
    try:
        Lat = g.osm['y']
    except TypeError:
        Lat = 0
    
    Latitude.append(Lat)
    
    try:
        Lon = g.osm['x']
    except TypeError:
        Lon = 0
        
    Longitude.append(Lon)
    
    j+=1

Latitude, Longitude

([53.6023971,
  53.6374003,
  53.6284111,
  53.6633,
  53.6323399,
  53.6215437,
  53.6105412,
  53.6086,
  53.5963901,
  53.5903912,
  53.5724835,
  53.5754331,
  53.5684065,
  53.5539347,
  53.5498808,
  53.55049605,
  53.5393237,
  53.5196541,
  53.5308822,
  53.5569933,
  53.5550288,
  53.5468152,
  53.548722,
  53.5285492,
  53.5254239,
  53.5355872,
  53.5367196,
  53.5275068,
  53.561882,
  53.5672366,
  53.5534434,
  53.5540624,
  53.5488992,
  53.5873864,
  53.5825665,
  53.5715093,
  53.58646755,
  53.5550658,
  53.5690699,
  53.605,
  53.5967771,
  53.5655862,
  53.5550626,
  53.5524166,
  53.5575,
  53.5814103,
  53.5847,
  53.5738475,
  53.5929519,
  53.4922921,
  53.5017484,
  53.4757009,
  53.4541741,
  53.4637464,
  53.4513998,
  53.4455855,
  53.4394131,
  53.4367554,
  0,
  53.4245599,
  53.4672,
  53.4493851,
  53.4848635,
  53.5040949,
  53.5047,
  53.484249,
  53.469968,
  53.537285,
  53.5182412,
  53.5063478,
  53.4858,
  53.4297246,
  53.4490678,
  53.4206766,
 

### 3. produce dataframe

In [11]:
# produce dataframe
dictTemp = {'Stadtteile': Stadtteile, 'Latitude' : Latitude, 'Longitude' : Longitude}
dfST = pd.DataFrame.from_dict(dictTemp)
dfST.head()

,Stadtteile,Latitude,Longitude
0,Lokstedt,53.602397,9.956662
1,Schnelsen,53.637400,9.918831
2,Niendorf,53.628411,9.955709
3,Langenhorn,53.663300,10.008300
4,Fuhlsbüttel,53.632340,10.023369


In [12]:
print(dfST.dtypes)

Stadtteile     object
Latitude      float64
Longitude     float64
dtype: object


In [13]:
# check for missing coordinates
dfST.loc[dfST['Latitude'] == 0]

,Stadtteile,Latitude,Longitude
58,Langenbeck,0.0,0.0


In [14]:
#Langenbeck turns out to have been misspelled on the Wiki page

g = geocoder.osm('Langenbek, Hamburg')
g.osm

{'x': 9.9893107,
 'y': 53.4321415,
 'addr:state': 'Hamburg',
 'addr:country': 'Deutschland'}

In [15]:
# change name and add values to dataframe
dfST.iloc[58:59, 0] = 'Langenbek'
dfST.iloc[58:59, 1] = g.osm['y']
dfST.iloc[58:59, 2] = g.osm['x']

dfST.iloc[58:59]

,Stadtteile,Latitude,Longitude
58,Langenbek,53.432142,9.989311


Now generate a map to check everything is fine 

In [16]:
#from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium
from pandas.io.json import json_normalize

In [17]:
address = 'Hamburg, Germany'

g = geocoder.osm(address)

#location = geolocator.geocode(address)
latitudeHH = g.osm['y']
longitudeHH = g.osm['x']
print('The geograpical coordinates of Hamburg are {}, {}.'.format(latitudeHH, longitudeHH))

The geograpical coordinates of Hamburg are 53.550341, 10.000654.


In [18]:
# create map of Hamburg using latitude and longitude values
map = folium.Map(location=[latitudeHH, longitudeHH], zoom_start=13)

# add markers for every neighborhood to map
for lat, lng, ST in zip(dfST['Latitude'], dfST['Longitude'], dfST['Stadtteile']):
    label = '{}'.format(ST)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=2,
        popup=label,
        color='green',
        fill=True,
        #fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map)  
    
map

## Scrape Foursquare

Now check if there are any gluten-free restaurants and see where they are located.

In [69]:
CLIENT_ID = '----' # your Foursquare ID
CLIENT_SECRET = '----' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 50

In [70]:
search_query = 'gluten-free'

category = '4d4b7105d754a06374d81259' #That is the categoryID for 'food', which includes all kinds of restaurants, bars etc.
radius = 10000
Limit = 50

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&intent=browse&categoryId={}&ll={},{}&v={}&query={}&radius={}'.format(CLIENT_ID, CLIENT_SECRET, category, latitudeHH, longitudeHH, VERSION, search_query, radius)

url

'https://api.foursquare.com/v2/venues/search?client_id=----&client_secret=----&intent=browse&categoryId=4d4b7105d754a06374d81259&ll=53.550341,10.000654&v=20180604&query=gluten-free&radius=10000'

In [21]:
# let's see what we've got

result = requests.get(url).json()

result

{'meta': {'code': 200, 'requestId': '5d9ca846f96b2c002c84e9f8'},
 'response': {'venues': [{'id': '5a9e4acf112c6c0f8bfa1be1',
    'name': 'Isabella Glutenfreie Pâtisserie',
    'location': {'address': 'Stephansplatz 1-3',
     'lat': 53.557695,
     'lng': 9.988113,
     'labeledLatLngs': [{'label': 'display',
       'lat': 53.557695,
       'lng': 9.988113}],
     'distance': 1165,
     'cc': 'DE',
     'country': 'Deutschland',
     'formattedAddress': ['Stephansplatz 1-3', 'Deutschland']},
    'categories': [{'id': '5744ccdfe4b0c0459246b4e2',
      'name': 'Pastry Shop',
      'pluralName': 'Pastry Shops',
      'shortName': 'Pastry',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/dessert_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1570547782',
    'hasPerk': False},
   {'id': '50267318e4b064c42c1f4591',
    'name': 'Kiosk Öjendorfer Park',
    'location': {'address': 'Barsbütteler Weg',
     'crossStreet': 'Haßloredder',
     'la

In [22]:
# assign relevant part of JSON to venues
venues = result['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()


,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.cc,location.country,location.formattedAddress,location.crossStreet,location.postalCode,location.city,location.state
0,5a9e4acf112c6c0f8bfa1be1,Isabella Glutenfreie Pâtisserie,"[{'id': '5744ccdfe4b0c0459246b4e2', 'name': 'P...",v-1570547782,False,Stephansplatz 1-3,53.557695,9.988113,"[{'label': 'display', 'lat': 53.557695, 'lng':...",1165,DE,Deutschland,"[Stephansplatz 1-3, Deutschland]",NaN,NaN,NaN,NaN
1,50267318e4b064c42c1f4591,Kiosk Öjendorfer Park,"[{'id': '4d954b0ea243a5684a65b473', 'name': 'C...",v-1570547782,False,Barsbütteler Weg,53.560252,10.147037,"[{'label': 'display', 'lat': 53.56025201780851...",9742,DE,Deutschland,"[Barsbütteler Weg (Haßloredder), 22117 Hamburg...",Haßloredder,22117,Hamburg,Hamburg
2,5a05b48a4382ab47102ee211,Glut‘N‘Free,"[{'id': '4bf58dd8d48988d118951735', 'name': 'G...",v-1570547782,False,NaN,53.551229,9.931075,"[{'label': 'display', 'lat': 53.551229, 'lng':...",4602,DE,Deutschland,[Deutschland],NaN,NaN,NaN,NaN


#### So, just three places!
That's unsatisfying. Let's have a look where and what they are anyway.

In [23]:
# define the dataframe columns to put the results in
column_names = ['PostalCode', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

neighborhoods

,PostalCode,Latitude,Longitude


In [24]:
# Then let's loop through the data and fill the dataframe one row at a time.

for data in dataframe:
    postalcode = ['postalCode']
    neighborhood_lat = ['lat']
    neighborhood_lon = ['lon']
    
    neighborhoods = neighborhoods.append({'PostalCode': postalcode,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

# dataframe = dataframe[dataframe.id == category]
    
dataframe.head()

,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.cc,location.country,location.formattedAddress,location.crossStreet,location.postalCode,location.city,location.state
0,5a9e4acf112c6c0f8bfa1be1,Isabella Glutenfreie Pâtisserie,"[{'id': '5744ccdfe4b0c0459246b4e2', 'name': 'P...",v-1570547782,False,Stephansplatz 1-3,53.557695,9.988113,"[{'label': 'display', 'lat': 53.557695, 'lng':...",1165,DE,Deutschland,"[Stephansplatz 1-3, Deutschland]",NaN,NaN,NaN,NaN
1,50267318e4b064c42c1f4591,Kiosk Öjendorfer Park,"[{'id': '4d954b0ea243a5684a65b473', 'name': 'C...",v-1570547782,False,Barsbütteler Weg,53.560252,10.147037,"[{'label': 'display', 'lat': 53.56025201780851...",9742,DE,Deutschland,"[Barsbütteler Weg (Haßloredder), 22117 Hamburg...",Haßloredder,22117,Hamburg,Hamburg
2,5a05b48a4382ab47102ee211,Glut‘N‘Free,"[{'id': '4bf58dd8d48988d118951735', 'name': 'G...",v-1570547782,False,NaN,53.551229,9.931075,"[{'label': 'display', 'lat': 53.551229, 'lng':...",4602,DE,Deutschland,[Deutschland],NaN,NaN,NaN,NaN


In [25]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
  
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered.head()

,name,categories,address,lat,lng,labeledLatLngs,distance,cc,country,formattedAddress,crossStreet,postalCode,city,state,id
0,Isabella Glutenfreie Pâtisserie,Pastry Shop,Stephansplatz 1-3,53.557695,9.988113,"[{'label': 'display', 'lat': 53.557695, 'lng':...",1165,DE,Deutschland,"[Stephansplatz 1-3, Deutschland]",NaN,NaN,NaN,NaN,5a9e4acf112c6c0f8bfa1be1
1,Kiosk Öjendorfer Park,Convenience Store,Barsbütteler Weg,53.560252,10.147037,"[{'label': 'display', 'lat': 53.56025201780851...",9742,DE,Deutschland,"[Barsbütteler Weg (Haßloredder), 22117 Hamburg...",Haßloredder,22117,Hamburg,Hamburg,50267318e4b064c42c1f4591
2,Glut‘N‘Free,Grocery Store,NaN,53.551229,9.931075,"[{'label': 'display', 'lat': 53.551229, 'lng':...",4602,DE,Deutschland,[Deutschland],NaN,NaN,NaN,NaN,5a05b48a4382ab47102ee211


In [26]:
venues_map = folium.Map(location=[latitudeHH, longitudeHH], zoom_start=12) # generate map centred around the Hamburg center

# add neighborhood markers to map
for lat, lng, ST in zip(dfST['Latitude'], dfST['Longitude'], dfST['Stadtteile']):
    label = '{}'.format(ST)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='green',
        fill=True,
        fill_color='green',
        fill_opacity=0.6,
        parse_html=False).add_to(venues_map)
    
# add a red circle marker to represent Hamburg center
folium.features.CircleMarker(
    [latitudeHH, longitudeHH],
    radius=5,
    color='red',
    popup='Center',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
    ).add_to(venues_map)

# add the restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

venues_map

So, there is a restaurant somewhere around the city center and a dedicated gluten-free grocery shop quite a drive from the center. The convenience store is almost not Hamburg anymore... 
Poor gluten sensitive people!

Just for fun, let's check New York...

In [27]:
# Lets make a dataframe with the three central neighborhoods and see where restaurants are in general.

dfNew = pd.DataFrame()

df0 = dfST[dfST['Stadtteile'] == 'Altstadt']
df1 = dfST[dfST['Stadtteile'] == 'Neustadt']
df2 = dfST[dfST['Stadtteile'] == "St. Georg"]

dfNew = pd.concat([df0, df1, df2], ignore_index = True)
dfNew

,Stadtteile,Latitude,Longitude
0,Altstadt,53.550496,9.997776
1,Neustadt,53.549881,9.979048
2,St. Georg,53.556993,10.014162


In [28]:
category = '4d4b7105d754a06374d81259'
radius = 500
i=0
resultNew = {}
dfMap = pd.DataFrame()

while i<3:
    Stadtteil = dfNew.loc[i,'Stadtteile']
    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&intent=browse&categoryId={}&near={}+Hamburg&v={}&radius={}'.format(CLIENT_ID, CLIENT_SECRET, category, Stadtteil, VERSION, radius)
    resultsNew = requests.get(url).json()
    venuesNew = resultsNew['response']['venues']
    dataframeNew = json_normalize(venuesNew)
    for data in dataframeNew:
        postalcode = ['postalCode']
        neighborhood_lat = ['lat']
        neighborhood_lon = ['lon']
    
        neighborhoods = neighborhoods.append({'PostalCode': postalcode,
                                              'Latitude': neighborhood_lat,
                                              'Longitude': neighborhood_lon}, ignore_index=True)
    
    filtered_columns = ['name', 'categories'] + [col for col in dataframeNew.columns if col.startswith('location.')] + ['id']
    dataframe_filteredNew = dataframeNew.loc[:, filtered_columns]
    dataframe_filteredNew['categories'] = dataframe_filteredNew.apply(get_category_type, axis=1)
    dataframe_filteredNew.columns = [column.split('.')[-1] for column in dataframe_filteredNew.columns]
#concatenate dataframes
    dfMap = pd.concat([dfMap, dataframe_filteredNew], ignore_index = True, sort = False)
    i += 1
    
dfMap.drop_duplicates(subset = ['name'], keep = False)
                        
              
venues_map1 = folium.Map(location=[latitudeHH, longitudeHH], zoom_start=14) # generate map centred around the Hamburg center

# add neighborhood markers to map
for lat, lng, ST in zip(dfNew['Latitude'], dfNew['Longitude'], dfNew['Stadtteile']):
    label = '{}'.format(ST)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=2,
        popup=label,
        color='green',
        fill=True,
        fill_color='green',
        fill_opacity=0.6,
        parse_html=False).add_to(venues_map1)
    
# add the gluten-free restaurants as red circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=2,
        color='red',
        popup=label,
        fill = True,
        fill_color='red',
        fill_opacity=0.6
    ).add_to(venues_map1)

# add the restaurants as blue circle markers
for lat, lng, label in zip(dfMap.lat, dfMap.lng, dfMap.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=2,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map1)



venues_map1

In [29]:
#At least check what users say about the Pastry Shop
vid = dataframe.loc[0, 'id'] 

urlt = 'https://api.foursquare.com/v2/venues/{}?&client_id={}&client_secret={}&v={}'.format(vid, CLIENT_ID, CLIENT_SECRET, VERSION)
urlt

'https://api.foursquare.com/v2/venues/5a9e4acf112c6c0f8bfa1be1?&client_id=IHZMAHA2PTH0FPMO2NOQIYPFJISJ22U3FXUH21INKYLQDY5W&client_secret=BYGYV5KW2GD3OLLXD2QHLMJAOY4DKX4ZVJBISMTK5M4B1CM2&v=20180604'

In [30]:
result_t = requests.get(urlt).json()
result_t

{'meta': {'code': 200, 'requestId': '5d9ca84f446ea6002cbadfa8'},
 'response': {'venue': {'id': '5a9e4acf112c6c0f8bfa1be1',
   'name': 'Isabella Glutenfreie Pâtisserie',
   'contact': {},
   'location': {'address': 'Stephansplatz 1-3',
    'lat': 53.557695,
    'lng': 9.988113,
    'labeledLatLngs': [{'label': 'display',
      'lat': 53.557695,
      'lng': 9.988113}],
    'cc': 'DE',
    'country': 'Deutschland',
    'formattedAddress': ['Stephansplatz 1-3', 'Deutschland']},
   'canonicalUrl': 'https://foursquare.com/v/isabella-glutenfreie-p%C3%A2tisserie/5a9e4acf112c6c0f8bfa1be1',
   'categories': [{'id': '5744ccdfe4b0c0459246b4e2',
     'name': 'Pastry Shop',
     'pluralName': 'Pastry Shops',
     'shortName': 'Pastry',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/dessert_',
      'suffix': '.png'},
     'primary': True}],
   'verified': False,
   'stats': {'tipCount': 2},
   'likes': {'count': 3,
    'groups': [{'type': 'others',
      'count': 3,
      'ite

In [31]:
tipcount = result_t['response']['venue']['stats']
tips = result_t['response']['venue']['tips']['groups'][0]['items'][0]['text']
print(tipcount, tips)


{'tipCount': 2} Great coffee and awesome glutenfree pastry and bread! #bestplacetogo


Well, Germans are not that into Foursqure it seems... just two tips and one is actually empty.

Let's see if we can cluster the overall "food" lendscape in central Hamburg.

In [32]:
print('There are {} uniques categories.'.format(len(dfMap['categories'].unique())))

There are 44 uniques categories.


In [33]:
dfMap

,name,categories,address,lat,lng,labeledLatLngs,postalCode,cc,city,state,country,formattedAddress,crossStreet,neighborhood,id
0,Public Coffee Roasters,Coffee Shop,Brandstwiete 3,53.546953,9.997440,"[{'label': 'display', 'lat': 53.54695300323186...",20457,DE,Hamburg,Hamburg,Deutschland,"[Brandstwiete 3, 20457 Hamburg, Deutschland]",NaN,NaN,59aaaf194a1cc02d7a736c6a
1,bona'me,Pide Place,Burchardstr. 17,53.548555,10.000944,"[{'label': 'display', 'lat': 53.54855549896045...",20095,DE,Hamburg,Hamburg,Deutschland,"[Burchardstr. 17, 20095 Hamburg, Deutschland]",NaN,NaN,58922d0618384c317754caeb
2,Café Paris,Café,Rathausstr. 4,53.550106,9.994227,"[{'label': 'display', 'lat': 53.55010564784363...",20095,DE,Hamburg,Hamburg,Deutschland,"[Rathausstr. 4, 20095 Hamburg, Deutschland]",NaN,NaN,4b011362f964a520ab4122e3
3,Hofbräu Wirtshaus am Speersort,Bavarian Restaurant,Speersort 1,53.549546,9.997839,"[{'label': 'display', 'lat': 53.54954609888491...",20095,DE,Hamburg,Hamburg,Deutschland,"[Speersort 1 (Domstr.), 20095 Hamburg, Deutsch...",Domstr.,NaN,4e748542e4cd325436dfd642
4,Food Sky,Food Court,Ballindamm 40,53.551358,9.996600,"[{'label': 'display', 'lat': 53.55135797384677...",20095,DE,Hamburg,Hamburg,Deutschland,"[Ballindamm 40, 20095 Hamburg, Deutschland]",NaN,Hamburg-Altstadt,59f343a03b83077c09110535
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,Bus Stop,Café,NaN,53.552012,10.012107,"[{'label': 'display', 'lat': 53.55201207234733...",NaN,DE,Hamburg,Hamburg,Deutschland,"[Hamburg, Deutschland]",NaN,NaN,58c56b0e1d21ba792e92ec98
86,Pho Duc,Vietnamese Restaurant,Steindamm 103,53.556740,10.019367,"[{'label': 'display', 'lat': 53.55673974621619...",20099,DE,Hamburg,Hamburg,Deutschland,"[Steindamm 103, 20099 Hamburg, Deutschland]",NaN,NaN,4b5719ecf964a520712628e3
87,Baumann's Bierbar,Bar,Kirchenallee 54,53.553066,10.008584,"[{'label': 'display', 'lat': 53.55306600458851...",20099,DE,Hamburg,Hamburg,Deutschland,"[Kirchenallee 54, 20099 Hamburg, Deutschland]",NaN,NaN,4bd9e4652a3a0f4791c0a8b6
88,dean&david,Salad Place,Hachmannplatz 16,53.552463,10.007254,"[{'label': 'display', 'lat': 53.55246270781142...",20099,DE,Hamburg,Hamburg,Deutschland,"[Hachmannplatz 16 (Südsteg), 20099 Hamburg, De...",Südsteg,NaN,572368b0cd1051829a9374f6


In [34]:
#fill the missing neighborhoods
g = geocoder.osm('Neustadt, Hamburg')
g.osm

{'x': 9.9790482,
 'y': 53.5498808,
 'addr:city': 'Hamburg',
 'addr:state': 'Hamburg',
 'addr:country': 'Deutschland'}

In [35]:
# clean the dataframe
dfMap.dropna(subset = ['postalCode'], inplace = True)

In [36]:
# one hot encoding
centralHH_onehot = pd.get_dummies(dfMap[['categories']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
centralHH_onehot['PLZ'] = dfMap['postalCode'] 

# move PLZ column to the first column
fixed_columns = [centralHH_onehot.columns[-1]] + list(centralHH_onehot.columns[:-1])
centralHH_onehot = centralHH_onehot[fixed_columns]

centralHH_onehot

,PLZ,BBQ Joint,Bakery,Bar,Bavarian Restaurant,Beer Bar,Brewery,Burger Joint,Café,Chinese Restaurant,...,Seafood Restaurant,Shopping Mall,Spanish Restaurant,Steakhouse,Swabian Restaurant,Thai Restaurant,Trattoria/Osteria,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
0,20457,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,20095,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,20095,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,20095,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,20095,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,20095,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
86,20099,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
87,20099,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
88,20099,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [37]:
grouped = centralHH_onehot.groupby('PLZ').mean().reset_index()
grouped

,PLZ,BBQ Joint,Bakery,Bar,Bavarian Restaurant,Beer Bar,Brewery,Burger Joint,Café,Chinese Restaurant,...,Seafood Restaurant,Shopping Mall,Spanish Restaurant,Steakhouse,Swabian Restaurant,Thai Restaurant,Trattoria/Osteria,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
0,20095,0.037037,0.037037,0.000000,0.037037,0.000000,0.0,0.074074,0.037037,0.000000,...,0.000000,0.037037,0.037037,0.037037,0.000000,0.037037,0.037037,0.000000,0.000000,0.000000
1,20097,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,20099,0.047619,0.047619,0.047619,0.000000,0.000000,0.0,0.095238,0.047619,0.047619,...,0.047619,0.000000,0.000000,0.047619,0.000000,0.000000,0.000000,0.047619,0.000000,0.047619
3,20354,0.000000,0.066667,0.000000,0.000000,0.000000,0.0,0.066667,0.066667,0.000000,...,0.000000,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,20355,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.142857,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.142857,0.000000,0.000000,0.142857,0.142857,0.000000
5,20457,0.000000,0.100000,0.000000,0.000000,0.000000,0.2,0.100000,0.000000,0.000000,...,0.100000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.100000
6,20459,0.000000,0.000000,0.000000,0.000000,0.333333,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000


In [61]:
# create a function to sort venues and put the 10 most abundent into a new dataframe
import numpy as np
num_top_venues = 10
indicators = ['st', 'nd', 'rd']

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

# create columns according to number of top venues
columns = ['PLZ']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['PLZ'] = grouped['PLZ']

for ind in np.arange(grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,PLZ,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,20095,Coffee Shop,Fast Food Restaurant,German Restaurant,Burger Joint,Italian Restaurant,Pide Place,Bakery,Bavarian Restaurant,Café,Comfort Food Restaurant
1,20097,Persian Restaurant,Vietnamese Restaurant,Fried Chicken Joint,Food Court,Fast Food Restaurant,Falafel Restaurant,Doner Restaurant,Deli / Bodega,Currywurst Joint,Comfort Food Restaurant
2,20099,German Restaurant,Burger Joint,Coffee Shop,Pub,Bakery,Bar,Café,Chinese Restaurant,Doner Restaurant,Falafel Restaurant
3,20354,Coffee Shop,French Restaurant,Currywurst Joint,Bakery,Steakhouse,Deli / Bodega,Burger Joint,Café,Salad Place,Italian Restaurant
4,20355,Rhenisch Restaurant,Coffee Shop,Turkish Restaurant,Vegetarian / Vegan Restaurant,Swabian Restaurant,Italian Restaurant,Café,Falafel Restaurant,Doner Restaurant,Deli / Bodega


## Cluster neigborhoods (PLZ-based)

In [62]:
# set number of clusters
kclusters = 6

grouped_clustering = grouped.drop('PLZ', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 2, 1, 5, 0, 3, 4])

In [63]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'ClusterLabel', kmeans.labels_)

merged = dfMap

# merge grouped with manhattan_data to add latitude/longitude for each neighborhood
merged = merged.join(neighborhoods_venues_sorted.set_index('PLZ'), on='postalCode')

merged.head() # check the last columns!

,name,categories,address,lat,lng,labeledLatLngs,postalCode,cc,city,state,...,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Public Coffee Roasters,Coffee Shop,Brandstwiete 3,53.546953,9.997440,"[{'label': 'display', 'lat': 53.54695300323186...",20457,DE,Hamburg,Hamburg,...,Brewery,Coffee Shop,Vietnamese Restaurant,Bakery,Hotel,Italian Restaurant,Burger Joint,Seafood Restaurant,Currywurst Joint,Fast Food Restaurant
1,bona'me,Pide Place,Burchardstr. 17,53.548555,10.000944,"[{'label': 'display', 'lat': 53.54855549896045...",20095,DE,Hamburg,Hamburg,...,Coffee Shop,Fast Food Restaurant,German Restaurant,Burger Joint,Italian Restaurant,Pide Place,Bakery,Bavarian Restaurant,Café,Comfort Food Restaurant
2,Café Paris,Café,Rathausstr. 4,53.550106,9.994227,"[{'label': 'display', 'lat': 53.55010564784363...",20095,DE,Hamburg,Hamburg,...,Coffee Shop,Fast Food Restaurant,German Restaurant,Burger Joint,Italian Restaurant,Pide Place,Bakery,Bavarian Restaurant,Café,Comfort Food Restaurant
3,Hofbräu Wirtshaus am Speersort,Bavarian Restaurant,Speersort 1,53.549546,9.997839,"[{'label': 'display', 'lat': 53.54954609888491...",20095,DE,Hamburg,Hamburg,...,Coffee Shop,Fast Food Restaurant,German Restaurant,Burger Joint,Italian Restaurant,Pide Place,Bakery,Bavarian Restaurant,Café,Comfort Food Restaurant
4,Food Sky,Food Court,Ballindamm 40,53.551358,9.996600,"[{'label': 'display', 'lat': 53.55135797384677...",20095,DE,Hamburg,Hamburg,...,Coffee Shop,Fast Food Restaurant,German Restaurant,Burger Joint,Italian Restaurant,Pide Place,Bakery,Bavarian Restaurant,Café,Comfort Food Restaurant


In [64]:
# create map
map_clusters = folium.Map(location=[latitudeHH, longitudeHH], zoom_start=14)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(merged['lat'], merged['lng'], merged['postalCode'], merged['ClusterLabel']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [65]:
#Cluster 1
merged.loc[merged['ClusterLabel'] == 0, merged.columns[[1] + list(range(5, merged.shape[1]))]]

,categories,labeledLatLngs,postalCode,cc,city,state,country,formattedAddress,crossStreet,neighborhood,...,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
35,Coffee Shop,"[{'label': 'display', 'lat': 53.551372, 'lng':...",20355,DE,Hamburg,Hamburg,Deutschland,"[Wexstr. 28, 20355 Hamburg, Deutschland]",NaN,NaN,...,Rhenisch Restaurant,Coffee Shop,Turkish Restaurant,Vegetarian / Vegan Restaurant,Swabian Restaurant,Italian Restaurant,Café,Falafel Restaurant,Doner Restaurant,Deli / Bodega
40,Vegetarian / Vegan Restaurant,"[{'label': 'display', 'lat': 53.55552820181528...",20355,DE,Hamburg,Hamburg,Deutschland,"[Caffamacherreihe 49, 20355 Hamburg, Deutschland]",NaN,NaN,...,Rhenisch Restaurant,Coffee Shop,Turkish Restaurant,Vegetarian / Vegan Restaurant,Swabian Restaurant,Italian Restaurant,Café,Falafel Restaurant,Doner Restaurant,Deli / Bodega
42,Swabian Restaurant,"[{'label': 'display', 'lat': 53.55103259175023...",20355,DE,Hamburg,Hamburg,Deutschland,"[Wexstr. 31, 20355 Hamburg, Deutschland]",NaN,NaN,...,Rhenisch Restaurant,Coffee Shop,Turkish Restaurant,Vegetarian / Vegan Restaurant,Swabian Restaurant,Italian Restaurant,Café,Falafel Restaurant,Doner Restaurant,Deli / Bodega
44,Rhenisch Restaurant,"[{'label': 'display', 'lat': 53.5495920486824,...",20355,DE,Hamburg,Hamburg,Deutschland,"[Stadthausbrücke 1-3, 20355 Hamburg, Deutschland]",NaN,NaN,...,Rhenisch Restaurant,Coffee Shop,Turkish Restaurant,Vegetarian / Vegan Restaurant,Swabian Restaurant,Italian Restaurant,Café,Falafel Restaurant,Doner Restaurant,Deli / Bodega
50,Italian Restaurant,"[{'label': 'display', 'lat': 53.55109658695257...",20355,DE,Hamburg,Hamburg,Deutschland,"[Wexstr. 29, 20355 Hamburg, Deutschland]",NaN,NaN,...,Rhenisch Restaurant,Coffee Shop,Turkish Restaurant,Vegetarian / Vegan Restaurant,Swabian Restaurant,Italian Restaurant,Café,Falafel Restaurant,Doner Restaurant,Deli / Bodega
51,Turkish Restaurant,"[{'label': 'display', 'lat': 53.55066673466425...",20355,DE,Hamburg,Hamburg,Deutschland,"[Steinwegpassage 6, 20355 Hamburg, Deutschland]",NaN,NaN,...,Rhenisch Restaurant,Coffee Shop,Turkish Restaurant,Vegetarian / Vegan Restaurant,Swabian Restaurant,Italian Restaurant,Café,Falafel Restaurant,Doner Restaurant,Deli / Bodega
59,Café,"[{'label': 'display', 'lat': 53.55426200275170...",20355,DE,Hamburg,Hamburg,Deutschland,"[Poolstr. 32, 20355 Hamburg, Deutschland]",NaN,NaN,...,Rhenisch Restaurant,Coffee Shop,Turkish Restaurant,Vegetarian / Vegan Restaurant,Swabian Restaurant,Italian Restaurant,Café,Falafel Restaurant,Doner Restaurant,Deli / Bodega


This cluster looks mostly Rhenisch and Vegetarian

In [66]:
#Cluster 2
merged.loc[merged['ClusterLabel'] == 1, merged.columns[[1] + list(range(5, merged.shape[1]))]]

,categories,labeledLatLngs,postalCode,cc,city,state,country,formattedAddress,crossStreet,neighborhood,...,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Pide Place,"[{'label': 'display', 'lat': 53.54855549896045...",20095,DE,Hamburg,Hamburg,Deutschland,"[Burchardstr. 17, 20095 Hamburg, Deutschland]",NaN,NaN,...,Coffee Shop,Fast Food Restaurant,German Restaurant,Burger Joint,Italian Restaurant,Pide Place,Bakery,Bavarian Restaurant,Café,Comfort Food Restaurant
2,Café,"[{'label': 'display', 'lat': 53.55010564784363...",20095,DE,Hamburg,Hamburg,Deutschland,"[Rathausstr. 4, 20095 Hamburg, Deutschland]",NaN,NaN,...,Coffee Shop,Fast Food Restaurant,German Restaurant,Burger Joint,Italian Restaurant,Pide Place,Bakery,Bavarian Restaurant,Café,Comfort Food Restaurant
3,Bavarian Restaurant,"[{'label': 'display', 'lat': 53.54954609888491...",20095,DE,Hamburg,Hamburg,Deutschland,"[Speersort 1 (Domstr.), 20095 Hamburg, Deutsch...",Domstr.,NaN,...,Coffee Shop,Fast Food Restaurant,German Restaurant,Burger Joint,Italian Restaurant,Pide Place,Bakery,Bavarian Restaurant,Café,Comfort Food Restaurant
4,Food Court,"[{'label': 'display', 'lat': 53.55135797384677...",20095,DE,Hamburg,Hamburg,Deutschland,"[Ballindamm 40, 20095 Hamburg, Deutschland]",NaN,Hamburg-Altstadt,...,Coffee Shop,Fast Food Restaurant,German Restaurant,Burger Joint,Italian Restaurant,Pide Place,Bakery,Bavarian Restaurant,Café,Comfort Food Restaurant
5,Fast Food Restaurant,"[{'label': 'display', 'lat': 53.55140028466869...",20095,DE,Hamburg,Hamburg,Deutschland,"[Ballindamm 40, 20095 Hamburg, Deutschland]",NaN,Hamburg-Altstadt,...,Coffee Shop,Fast Food Restaurant,German Restaurant,Burger Joint,Italian Restaurant,Pide Place,Bakery,Bavarian Restaurant,Café,Comfort Food Restaurant
9,Coffee Shop,"[{'label': 'display', 'lat': 53.55125630451121...",20095,DE,Hamburg,Hamburg,Deutschland,"[Barkhof 3, 20095 Hamburg, Deutschland]",NaN,NaN,...,Coffee Shop,Fast Food Restaurant,German Restaurant,Burger Joint,Italian Restaurant,Pide Place,Bakery,Bavarian Restaurant,Café,Comfort Food Restaurant
10,Coffee Shop,"[{'label': 'display', 'lat': 53.55277, 'lng': ...",20095,DE,Hamburg,Hamburg,Deutschland,"[Kurze Mühren 4, 20095 Hamburg, Deutschland]",NaN,Hamburg-Altstadt,...,Coffee Shop,Fast Food Restaurant,German Restaurant,Burger Joint,Italian Restaurant,Pide Place,Bakery,Bavarian Restaurant,Café,Comfort Food Restaurant
11,German Restaurant,"[{'label': 'display', 'lat': 53.5488317399098,...",20095,DE,Hamburg,Hamburg,Deutschland,"[Brodschrangen 1-5, 20095 Hamburg, Deutschland]",NaN,Hamburg-Altstadt,...,Coffee Shop,Fast Food Restaurant,German Restaurant,Burger Joint,Italian Restaurant,Pide Place,Bakery,Bavarian Restaurant,Café,Comfort Food Restaurant
12,Burger Joint,"[{'label': 'display', 'lat': 53.549684, 'lng':...",20095,DE,Hamburg,Hamburg,Deutschland,"[Speersort 1, 20095 Hamburg, Deutschland]",NaN,NaN,...,Coffee Shop,Fast Food Restaurant,German Restaurant,Burger Joint,Italian Restaurant,Pide Place,Bakery,Bavarian Restaurant,Café,Comfort Food Restaurant
13,Fast Food Restaurant,"[{'label': 'display', 'lat': 53.5518788498287,...",20095,DE,Hamburg,Hamburg,Deutschland,"[Spitalerstr. 12, 20095 Hamburg, Deutschland]",NaN,NaN,...,Coffee Shop,Fast Food Restaurant,German Restaurant,Burger Joint,Italian Restaurant,Pide Place,Bakery,Bavarian Restaurant,Café,Comfort Food Restaurant


This cluster is mostly fast food and coffee shops

In [67]:
#Cluster 3
merged.loc[merged['ClusterLabel'] == 2, merged.columns[[1] + list(range(5, merged.shape[1]))]]

,categories,labeledLatLngs,postalCode,cc,city,state,country,formattedAddress,crossStreet,neighborhood,...,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
81,Persian Restaurant,"[{'label': 'display', 'lat': 53.55218509742974...",20097,DE,Hamburg,Hamburg,Deutschland,"[Adenauerallee 70, 20097 Hamburg, Deutschland]",NaN,St. Georg,...,Persian Restaurant,Vietnamese Restaurant,Fried Chicken Joint,Food Court,Fast Food Restaurant,Falafel Restaurant,Doner Restaurant,Deli / Bodega,Currywurst Joint,Comfort Food Restaurant


A persian restaurant

In [68]:
#Cluster 4
merged.loc[merged['ClusterLabel'] == 3, merged.columns[[1] + list(range(5, merged.shape[1]))]]

,categories,labeledLatLngs,postalCode,cc,city,state,country,formattedAddress,crossStreet,neighborhood,...,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Coffee Shop,"[{'label': 'display', 'lat': 53.54695300323186...",20457,DE,Hamburg,Hamburg,Deutschland,"[Brandstwiete 3, 20457 Hamburg, Deutschland]",NaN,NaN,...,Brewery,Coffee Shop,Vietnamese Restaurant,Bakery,Hotel,Italian Restaurant,Burger Joint,Seafood Restaurant,Currywurst Joint,Fast Food Restaurant
6,Vietnamese Restaurant,"[{'label': 'display', 'lat': 53.548291, 'lng':...",20457,DE,Hamburg,Hamburg,Deutschland,"[Kleine Reichenstr. 18, 20457 Hamburg, Deutsch...",NaN,NaN,...,Brewery,Coffee Shop,Vietnamese Restaurant,Bakery,Hotel,Italian Restaurant,Burger Joint,Seafood Restaurant,Currywurst Joint,Fast Food Restaurant
8,Seafood Restaurant,"[{'label': 'display', 'lat': 53.54991207017100...",20457,DE,Hamburg,Hamburg,Deutschland,"[Große Johannisstr. 3, 20457 Hamburg, Deutschl...",NaN,NaN,...,Brewery,Coffee Shop,Vietnamese Restaurant,Bakery,Hotel,Italian Restaurant,Burger Joint,Seafood Restaurant,Currywurst Joint,Fast Food Restaurant
20,Brewery,"[{'label': 'display', 'lat': 53.547205, 'lng':...",20457,DE,Hamburg,Hamburg,Deutschland,"[Willy-Brandt-Str. 47, 20457 Hamburg, Deutschl...",NaN,NaN,...,Brewery,Coffee Shop,Vietnamese Restaurant,Bakery,Hotel,Italian Restaurant,Burger Joint,Seafood Restaurant,Currywurst Joint,Fast Food Restaurant
26,Burger Joint,"[{'label': 'display', 'lat': 53.54866568794967...",20457,DE,Hamburg,Hamburg,Deutschland,"[Alter Fischmarkt 3, 20457 Hamburg, Deutschland]",NaN,NaN,...,Brewery,Coffee Shop,Vietnamese Restaurant,Bakery,Hotel,Italian Restaurant,Burger Joint,Seafood Restaurant,Currywurst Joint,Fast Food Restaurant
28,Italian Restaurant,"[{'label': 'display', 'lat': 53.54638647577128...",20457,DE,Hamburg,Hamburg,Deutschland,"[Brandstwiete 58, 20457 Hamburg, Deutschland]",NaN,NaN,...,Brewery,Coffee Shop,Vietnamese Restaurant,Bakery,Hotel,Italian Restaurant,Burger Joint,Seafood Restaurant,Currywurst Joint,Fast Food Restaurant
36,Coffee Shop,"[{'label': 'display', 'lat': 53.54869800850429...",20457,DE,Hamburg,Hamburg,Deutschland,"[Großer Burstah 50-52, 20457 Hamburg, Deutschl...",NaN,NaN,...,Brewery,Coffee Shop,Vietnamese Restaurant,Bakery,Hotel,Italian Restaurant,Burger Joint,Seafood Restaurant,Currywurst Joint,Fast Food Restaurant
37,Brewery,"[{'label': 'display', 'lat': 53.55053185163435...",20457,DE,Hamburg,Hamburg,Deutschland,"[Adolphsbrücke 7, 20457 Hamburg, Deutschland]",NaN,NaN,...,Brewery,Coffee Shop,Vietnamese Restaurant,Bakery,Hotel,Italian Restaurant,Burger Joint,Seafood Restaurant,Currywurst Joint,Fast Food Restaurant
47,Bakery,"[{'label': 'display', 'lat': 53.54858736475559...",20457,DE,Hamburg,Hamburg,Deutschland,"[Hahntrapp (Großer Burstah), 20457 Hamburg, De...",Großer Burstah,NaN,...,Brewery,Coffee Shop,Vietnamese Restaurant,Bakery,Hotel,Italian Restaurant,Burger Joint,Seafood Restaurant,Currywurst Joint,Fast Food Restaurant
58,Hotel,"[{'label': 'display', 'lat': 53.5495476966977,...",20457,DE,Hamburg,Hamburg,Deutschland,"[Alter Wall 40, 20457 Hamburg, Deutschland]",NaN,NaN,...,Brewery,Coffee Shop,Vietnamese Restaurant,Bakery,Hotel,Italian Restaurant,Burger Joint,Seafood Restaurant,Currywurst Joint,Fast Food Restaurant


Breweries and Coffe shops

# Conclusion

There is only one dedicated gluten-free restaurant in Hamburg. A city of over one milion people. Although further analysis would be necessary for a final decision, opening a gluten free restaurant in central Hamburg seems like a good investment. 